In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Download the data

In [51]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [52]:
concrete_data.shape

(1030, 9)

In [53]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [54]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [55]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [56]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [57]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [58]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [59]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## Build a Neural Network

In [60]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)

In [62]:
# build the model
model = regression_model()

## Train and Test the Network

In [63]:
# build the model
model.fit( X_test, y_test,epochs=50, verbose=2)

Epoch 1/50
10/10 - 0s - loss: 9560.4600
Epoch 2/50
10/10 - 0s - loss: 6733.0225
Epoch 3/50
10/10 - 0s - loss: 6179.3818
Epoch 4/50
10/10 - 0s - loss: 5573.0352
Epoch 5/50
10/10 - 0s - loss: 5198.0264
Epoch 6/50
10/10 - 0s - loss: 4848.5708
Epoch 7/50
10/10 - 0s - loss: 4529.4727
Epoch 8/50
10/10 - 0s - loss: 4190.4092
Epoch 9/50
10/10 - 0s - loss: 3927.1970
Epoch 10/50
10/10 - 0s - loss: 3657.9612
Epoch 11/50
10/10 - 0s - loss: 3417.0679
Epoch 12/50
10/10 - 0s - loss: 3188.0823
Epoch 13/50
10/10 - 0s - loss: 2981.8452
Epoch 14/50
10/10 - 0s - loss: 2800.9055
Epoch 15/50
10/10 - 0s - loss: 2612.7292
Epoch 16/50
10/10 - 0s - loss: 2461.4792
Epoch 17/50
10/10 - 0s - loss: 2312.5955
Epoch 18/50
10/10 - 0s - loss: 2176.4600
Epoch 19/50
10/10 - 0s - loss: 2051.5989
Epoch 20/50
10/10 - 0s - loss: 1948.2386
Epoch 21/50
10/10 - 0s - loss: 1835.7426
Epoch 22/50
10/10 - 0s - loss: 1732.5939
Epoch 23/50
10/10 - 0s - loss: 1641.7814
Epoch 24/50
10/10 - 0s - loss: 1563.9973
Epoch 25/50
10/10 - 0s - 

In [64]:
loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

10/10 [==============================] - 0s 1ms/step - loss: 350.5468


350.5467529296875

Create a list of mean squared errors and the standard deviation of the mean squared errors

In [65]:
total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))

MSE 1: 100.22285461425781
MSE 2: 73.92572021484375
MSE 3: 69.8863525390625
MSE 4: 70.10360717773438
MSE 5: 70.1104507446289
MSE 6: 71.40313720703125
MSE 7: 74.38875579833984
MSE 8: 73.67395782470703
MSE 9: 68.07734680175781
MSE 10: 67.53147888183594
MSE 11: 72.10446166992188
MSE 12: 74.71598052978516
MSE 13: 64.6994857788086
MSE 14: 63.64726638793945
MSE 15: 63.94511032104492
MSE 16: 63.32316207885742
MSE 17: 66.27095031738281
MSE 18: 65.89783477783203
MSE 19: 66.51173400878906
MSE 20: 62.85072708129883
MSE 21: 61.45245361328125
MSE 22: 61.93920135498047
MSE 23: 70.57374572753906
MSE 24: 60.48957061767578
MSE 25: 61.21920394897461
MSE 26: 74.91633605957031
MSE 27: 59.634552001953125
MSE 28: 63.50912094116211
MSE 29: 58.37479019165039
MSE 30: 58.7370491027832
MSE 31: 62.10032272338867
MSE 32: 56.39643859863281
MSE 33: 56.31528091430664
MSE 34: 59.3795051574707
MSE 35: 67.92854309082031
MSE 36: 62.88908386230469
MSE 37: 51.7027587890625
MSE 38: 51.729732513427734
MSE 39: 49.3063545227050

We see that both standard deviation and mean increase the performance when we work with normalized data